# Overall Steps

- Find percentage of empty values per dimension
- Reduce dimensionality based on relevance

In [3]:
import pandas as pd

#mpg = pd.read_csv("C:/Users/evana/source/repos/Syracuse_Grad/CIS_600_Fund_Data_Mining/HW1/employee_attrition.csv")
mpg = pd.read_csv("C:/Users/evana/Documents/GitHub/SyracuseMasters/CIS_600_Fund_Data_Mining/HW1/employee_attrition.csv")

mpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1176 non-null   int64  
 1   Attrition                 1176 non-null   object 
 2   BusinessTravel            1176 non-null   object 
 3   DailyRate                 1176 non-null   int64  
 4   Department                1176 non-null   object 
 5   DistanceFromHome          1174 non-null   float64
 6   Education                 1176 non-null   int64  
 7   EducationField            1176 non-null   object 
 8   EmployeeCount             1176 non-null   int64  
 9   EmployeeNumber            1176 non-null   int64  
 10  EnvironmentSatisfaction   1176 non-null   int64  
 11  Gender                    1175 non-null   object 
 12  HourlyRate                1176 non-null   int64  
 13  JobInvolvement            1176 non-null   int64  
 14  JobLevel

In [4]:
obj_col = mpg.select_dtypes('object').columns

dataCleaningCounts = []
for col in obj_col:
    dataCleaningCounts.append([col, mpg[col].nunique(), mpg[col].isnull().sum()])
    
# Create the pandas DataFrame
pd.DataFrame(dataCleaningCounts, columns = ['Column', 'Unique Values', 'Nulls'])

,Column,Unique Values,Nulls
0,Attrition,2,0
1,BusinessTravel,3,0
2,Department,3,0
3,EducationField,6,0
4,Gender,2,1
5,JobRole,9,0
6,MaritalStatus,3,0
7,Over18,1,0
8,OverTime,2,1
